In [10]:
from hello.hello3 import HelloApp, NotLoggedInError
from scripts.archive.analysis.mixingtime import CompiledMixingTime, CompiledMixingSaveAsInfo
import os
import re

path = "C:\\users\\natha\\documents\\personal\\test\\mixing_time_raw2\\"
mixing_test_re = re.compile(r"cond_c_(\d+)")
from pysrc import logger

logger = logger.BuiltinLogger(__name__)

def dl_all_batchnames(ipv4='192.168.1.14'):
    """
    download all batchnames from the given reactor and
    analyze them.
    """
    app = HelloApp(ipv4)
    app.login()
    batches = app.getBatches()
    ids = batches.ids_to_batches.keys()

    try:
        os.makedirs(path)
    except FileExistsError:
        pass

    for id in ids:
        name = batches.getbatchname(id)
        if name.startswith("cond_c_"):
            logger.info("Downloading Batch File: (%d) \"%s\"", id, name)
            report = app.getdatareport_bybatchid(id)

            with open(path + name + ".csv", 'wb') as f:
                f.write(report)

def analyze_batches():
    si = CompiledMixingSaveAsInfo(path.replace("_raw", ""))
    c = CompiledMixingTime(saveas_info=si)
    for file in os.listdir(path):
        c.add_csv(path + file)
    c.analyze_all()

def analyze_position_test():
    si = CompiledMixingSaveAsInfo(path.replace("_raw", "_positions"))
    c = CompiledMixingTime(saveas_info=si)
    for file in os.listdir(path):
        m = mixing_test_re.match(file)
        if not m:
            continue
        c.add_csv(path + file)
    c.analyze_all()

def analyze_mixing_time(files):
    si = CompiledMixingSaveAsInfo(path.replace("_raw", "_mixing"))
    c = CompiledMixingTime(saveas_info=si, logger=logger, tests_per_group=5)
    c.logger.setLevel(20)
    for file in files:
        m = mixing_test_re.match(file)
        if not m:
            continue
        c.add_csv(path + file, 'pHBRaw')
    c.sort_tests(key=lambda t: int(mixing_test_re.match(t.name).groups()[0]))
    c.analyze_all()

def main_161229():
    #dl_all_batchnames()
    analyze_mixing_time(os.listdir(path))

In [2]:
import time

def call(f, *args, **kw):
    s = "(" + ", ".join(map(repr, args)) + ")"
    m = "%s%s" % (f.__name__, s)
    #print(m)
    while True:
        try:
            rv = f(*args, **kw)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        except Exception: 
            print(str(e))
        else:
            return rv
        
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def endbatch():
    call(h.endbatch)
    
def runrecipe(name):
    call(h.runrecipe, name)

def start_test(num):
    global h
    h = HelloApp('192.168.1.14')
    call(h.login)
    endbatch()
    bn = "cond_c_%d" % (num)
    startbatch(bn)
    input("Press enter to start recipe")
    runrecipe("conductivity")
    input("Press enter when PV is stable")
    end = time.time() + 80
    print("Waiting 80 seconds. Press ctrl-c to cancel")
    try:
        while time.time() < end:
            time.sleep(0.1)
    except KeyboardInterrupt:
        pass
    call(h.login)
    endbatch()
    

In [3]:
i = 1

In [4]:
while True:
    start_test(i)
    i += 1
    b = input("Press enter to continue (or type \"quit\" and press enter to quit)")
    if b.lower() in ("q", "quit"):
        break

Press enter to start recipe


KeyboardInterrupt: 

In [ ]:
# app = HelloApp('192.168.1.15')
# app.login()
# for rpm in (12, 15, 19):
#     for i in range(1, 6):
#         bn = "mt %drpm 5.%d" % (rpm, i)
#         print("Downloading:", bn)
#         b = app.getdatareport_bybatchname(bn)
#         with open(path + "\\%s.csv" % bn, 'wb') as f:
#             f.write(b)

In [5]:
def download(num, app=None):
    call(h.login)
    bn = "cond_c_%d" % num
    print("Downloading %s"%bn)
    b = call(h.getdatareport_bybatchname, bn)
    with open(path + "\\%s.csv" % bn, 'wb') as f:
        f.write(b)

In [36]:
for i in range(2, 25):
#     if i == 15:
#         continue
    if i == 10:
        continue
    download(i)

In [9]:
analyze_mixing_time(os.listdir(path))

06/15/18 14:36:01 INFO <analyze_all>: Running analysis on 19 tests.
06/15/18 14:36:01 INFO <run_analysis>: Beginning analysis on: cond_c_6.csv
06/15/18 14:36:04 INFO <run_analysis>: Beginning analysis on: cond_c_7.csv
06/15/18 14:36:06 INFO <run_analysis>: Beginning analysis on: cond_c_8.csv
06/15/18 14:36:09 INFO <run_analysis>: Beginning analysis on: cond_c_9.csv
06/15/18 14:36:12 INFO <run_analysis>: Beginning analysis on: cond_c_10.csv
06/15/18 14:36:14 INFO <run_analysis>: Beginning analysis on: cond_c_11.csv
06/15/18 14:36:17 INFO <run_analysis>: Beginning analysis on: cond_c_12.csv
06/15/18 14:36:19 INFO <run_analysis>: Beginning analysis on: cond_c_13.csv
06/15/18 14:36:22 INFO <run_analysis>: Beginning analysis on: cond_c_14.csv
06/15/18 14:36:25 INFO <run_analysis>: Beginning analysis on: cond_c_15.csv
06/15/18 14:36:27 INFO <run_analysis>: Beginning analysis on: cond_c_16.csv
06/15/18 14:36:30 INFO <run_analysis>: Beginning analysis on: cond_c_17.csv
06/15/18 14:36:32 INFO <

In [ ]:
def main():
    import subprocess
    subprocess.call("tskill.exe excel")
    from time import sleep
    sleep(1)
    main_161229()

In [ ]:
def finish_formatting():
    from officelib.xllib import *
    for chart in xl.Selection:
        xlcom.FormatAxesScale(chart.Chart, None, 100)

    for i, co in enumerate(xl.Selection):
        row = i // 5
        col = i % 5
        co.Top = row * co.Height + 20
        co.Left = co.Width * col + 20